 # Построение корпуса текстов
 
 Тексты для корпуса взяты с сайта ["Астрономия для школьников"](https://astro.altspu.ru) из [раздела](https://astro.altspu.ru/events/words/word.html?word=185) новостей, содержащих ключевое слово "метеорит".  
 В конце каждой новости представлены ключевые слова. 

In [3]:
import urllib.request
import re
import os
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [26]:
url = 'https://astro.altspu.ru/events/words/word.html?word=185'
main_url = 'https://astro.altspu.ru'

In [166]:
# достаем ссылки на новости со страницы и добавляем их в список 
hrefs = []
response = requests.get(url)
soup = BeautifulSoup(response.text)
content = soup.find_all('div', {'id': 'content'})
for a in content[0].find_all('a'):
    href = a.attrs['href']
    if href.endswith('html') and href.startswith('/events'):
        hrefs.append(main_url + href)
hrefs = hrefs[::2]

In [259]:
df = pd.DataFrame(columns=['text', 'key_words'])

In [260]:
# достаем текст новости и ключевые слова со страницы
for href in hrefs:
    response = requests.get(href)
    soup = BeautifulSoup(response.text)
    
    text = soup.find_all('div', {'id': 'content'})[0].find_all('p')[0].text.split('Еще новости сходной тематики:')[0]
    text = re.sub('\n', '', text)
    
    if len(text) > 2000:
        key_words = re.search(r'(Ключевые слова: )(.+)( Разделы:)', 
                          soup.find_all('p', {'style': 'text-align:right;'})[0].text)
        if key_words:
            key_words = key_words.group(2)
        else:
            key_words = re.search(r'(Ключевые слова: )(.+)', soup.find_all('p')[-1].text).group(2)

        df = df.append({'text': text, 'key_words': key_words}, ignore_index=True)
    else:
        continue

In [265]:
# выделенные мной ключевые слова
text_0 = 'метеорит, Луна, гудение, сейсмическое колебание, '
text_1 = 'метеорит, взрыв, шар-болид, Тунгусский метеорит'
text_2 = 'метеорит, Луна, камушек, метеор'
text_3 = 'Марс, марсоход, метеорит, вода, Земля, Красная планета'
text_4 = 'метеорит, взрыв, Австралия, вспышка'

df['my_key_words'] = [text_0, text_1, text_2, text_3, text_4]

In [361]:
# эталон - объединения выделенных мной и данных ключевых слов
standard = []
for i in range(5):
    list1 = df['key_words'].to_list()[i].split(', ')
    list2 = df['my_key_words'].to_list()[i].split(', ')
    standard_key_words = list1 + list2
    standard.append(', '.join(set(standard_key_words)))
df['standard_key_words'] = standard

In [267]:
df

,text,key_words,my_key_words,standard_key_words
0,"Вибрации Луны, порождаемые постоянными ударам...","Луна, метеорит, бомбардировка, сейсмическая ак...","метеорит, Луна, гудение, сейсмическое колебание,",", гудение, сейсмическое колебание, сейсмическа..."
1,30 июня 1908 года около семи часов утра над о...,"метеорит, астрономия, история, Тунгусский мете...","метеорит, взрыв, шар-болид, Тунгусский метеорит","шар-болид, болид, астрономия, метеорит, взрыв,..."
2,Планам США создать на Луне постоянную базу к ...,"Луна, Smart-1, NASA, метеор, комета, метеорит,...","метеорит, Луна, камушек, метеор","комета, Smart-1, NASA, метеор, метеорит, лунна..."
3,Дэвид Шустер (David Shuster) из Калифорнийско...,"Марс, марсоход, вода, Красная планета, метеори...","Марс, марсоход, метеорит, вода, Земля, Красная...","вода, Земля, марсоход, метеорит, Марс, Красная..."
4,"Загадочная вспышка света и взрыв, разбудившие...",метеорит,"метеорит, взрыв, Австралия, вспышка","вспышка, метеорит , метеорит, взрыв, Австралия"


# Извлечение ключевых слов

In [314]:
from pymystem3 import Mystem
mystem = Mystem()

In [315]:
def normalize_text(text):
    lemmas = mystem.lemmatize(text)   
    return ' '.join(lemmas)

### RAKE

In [244]:
import RAKE
import nltk
from nltk.corpus import stopwords

In [245]:
st_words = stopwords.words('russian')

In [247]:
rake = RAKE.Rake(st_words)

In [316]:
rake_key_words = []
for text in df['text'].to_list():
    rake_kw = []
    for elem in rake.run(normalize_text(text), minFrequency=2, maxWords=2):
        rake_kw.append(elem[0])
    rake_key_words.append(', '.join(rake_kw))
df['rake_kw'] = rake_key_words

### TextRank

In [277]:
from summa import keywords

In [317]:
TextRank_key_words = []
for text in df['text'].to_list():
    tr_kw = keywords.keywords(normalize_text(text), language='russian', additional_stopwords=st_words).replace('\n', ', ')
    TextRank_key_words.append(tr_kw)
df['TextRank_kw'] = TextRank_key_words

### KeyBert

In [290]:
from keybert import KeyBERT

In [291]:
kw_model = KeyBERT('clips/mfaq')

Downloading:   0%|          | 0.00/1.65k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/3.74k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/778 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/294 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.08M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/464 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/transformers/configuration_utils.py:369: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [318]:
bert_key_words = []
for text in df['text'].to_list():
    bert_kw = []
    for elem in kw_model.extract_keywords(normalize_text(text), keyphrase_ngram_range=(1, 2), stop_words=st_words):
        bert_kw.append(elem[0])
    for elem in kw_model.extract_keywords(normalize_text(text), stop_words=st_words):
        bert_kw.append(elem[0])
    bert_key_words.append(', '.join(set(bert_kw)))

In [319]:
df['bert_kw'] = bert_key_words

## Шаблоны для ключевых слов и фраз

В качестве ключевых слов обычно выделяют существительные или именные группы (прилагательное + существительное), поэтому из всех ключевых слов отберем ключевые слова, соответсвующие этому "критерию".

In [322]:
from pprint import pprint

In [350]:
def filter_key_words(key_words):
    filtered_kw = []
    for element in key_words.lower().split(', '):
        kw_pos = []
        ana = m.analyze(element)
        for word in ana:
            if 'analysis' in word:
                if word['analysis'] == []:
                    pos = 'X'
                    kw_pos.append(pos)
                else:
                    gr = word['analysis'][0]['gr']
                    pos = gr.split('=')[0].split(',')[0]
                    kw_pos.append(pos)   
        if kw_pos == ['S'] or kw_pos == ['A','S']:
            filtered_kw.append(element)
    return filtered_kw

In [363]:
def make_list(string):
    kw_list = [word.lower() for word in string.split(', ') if word != '']
    return kw_list

In [351]:
df['filtered_standard_key_words'] = df['standard_key_words'].apply(filter_key_words)
df['filtered_rake_kw'] = df['rake_kw'].apply(filter_key_words)
df['filtered_TextRank_kw'] = df['TextRank_kw'].apply(filter_key_words)
df['filtered_bert_kw'] = df['bert_kw'].apply(filter_key_words)

In [365]:
df['standard_key_words'] = df['standard_key_words'].apply(make_list)
df['rake_kw'] = df['rake_kw'].apply(make_list)
df['TextRank_kw'] = df['TextRank_kw'].apply(make_list)
df['bert_kw'] = df['bert_kw'].apply(make_list)

In [366]:
df

,text,key_words,my_key_words,standard_key_words,rake_kw,TextRank_kw,bert_kw,filtered_rake_kw,filtered_standard_key_words,filtered_TextRank_kw,filtered_bert_kw
0,"Вибрации Луны, порождаемые постоянными ударам...","Луна, метеорит, бомбардировка, сейсмическая ак...","метеорит, Луна, гудение, сейсмическое колебание,","[гудение, сейсмическое колебание, сейсмическая...","[луна, поверхность, тысяча, оставлять, скоро]","[луна, лунный модуль, сейсмограф, ученый, апол...","[шум, внутри луна, луна появляться, метеоритны...","[луна, поверхность, тысяча]","[гудение, сейсмическое колебание, сейсмическая...","[луна, лунный модуль, сейсмограф, ученый, апол...","[шум, планета, волна, луна]"
1,30 июня 1908 года около семи часов утра над о...,"метеорит, астрономия, история, Тунгусский мете...","метеорит, взрыв, шар-болид, Тунгусский метеорит","[шар-болид, болид, астрономия, метеорит, взрыв...","[запад, это, версия]","[год около, выдвигать, экспедиция, тунгусский,...","[взрыв происходить, волна, утро обширный, взры...","[запад, версия]","[болид, астрономия, метеорит, взрыв, катаклизм...","[экспедиция, астроном, километр, писатель, исс...","[волна, взрыв, событие, пожар, катастрофа]"
2,Планам США создать на Луне постоянную базу к ...,"Луна, Smart-1, NASA, метеор, комета, метеорит,...","метеорит, Луна, камушек, метеор","[комета, smart-1, nasa, метеор, метеорит, лунн...","[вспышка, 0]","[луна, лунный, метеорит, метеор, энергия, набл...","[исследование земной, мочь помешать, сооружени...",[вспышка],"[комета, метеор, метеорит, лунная база , луна,...","[луна, метеорит, метеор, энергия, наблюдение, ...","[спутник, мочь, исследование, реальность]"
3,Дэвид Шустер (David Shuster) из Калифорнийско...,"Марс, марсоход, вода, Красная планета, метеори...","Марс, марсоход, метеорит, вода, Земля, Красная...","[вода, земля, марсоход, метеорит, марс, красна...","[марс, показывать, холодный, поверхность, земл...","[вода, марс, шустер, марсианский метеорит, пла...","[показывать марс, марс вероятно, марсоход, ист...","[марс, поверхность, земля]","[вода, земля, марсоход, метеорит, марс, красна...","[вода, марс, шустер, марсианский метеорит, пла...","[марсоход, планета, исследование, марс, история]"
4,"Загадочная вспышка света и взрыв, разбудившие...",метеорит,"метеорит, взрыв, Австралия, вспышка","[вспышка, метеорит , метеорит, взрыв, австралия]","[взрыв, ничто]","[который, житель, падение метеорит, взрыв, чел...","[светло день, разбудить, утро, волна, взрыв ра...",[взрыв],"[вспышка, метеорит , метеорит, взрыв, австралия]","[житель, взрыв, человек, ученый, специалист, в...","[утро, волна, взрыв, загадочный вспышка, небо]"


#  Оценка качества

## Без учета фильтра

## С учетом фильтра

In [371]:
def quality_metrics(pred_kw, true_kw):
    precision = len(set(pred_kw).intersection(true_kw)) / len(pred_kw)
    recall = len(set(pred_kw).intersection(true_kw)) / len(true_kw)
    if precision + recall != 0:
        f1 = 2 * precision * recall / (precision + recall)
    else:
        f1 = 0    
    return precision, recall, f1  

In [390]:
df_results = pd.DataFrame(columns=['method', 'precision', 'recall', 'f1_score'])

methods = ['rake_kw', 'TextRank_kw', 'bert_kw', 'filtered_rake_kw', 'filtered_TextRank_kw', 'filtered_bert_kw']
for method in methods:
    precision = 0
    recall = 0
    f1_score = 0
    
    pred_kw = df[method]
    
    if method.startswith('filtered'):
        true_kw = df['filtered_standard_key_words']
    else:
        true_kw = df['standard_key_words']
        
    for i in range(len(true_kw)):
        prec, rec, f1  = quality_metrics(pred_kw[i], true_kw[i])
        precision += prec
        recall += rec
        f1_score += f1
    df_results.loc[len(df_results.index)] = [method, precision/len(true_kw), recall/len(true_kw), f1_score/len(true_kw)]

In [391]:
df_results

,method,precision,recall,f1_score
0,rake_kw,0.206667,0.130476,0.155045
1,TextRank_kw,0.104874,0.422381,0.166445
2,bert_kw,0.102222,0.155476,0.122255
3,filtered_rake_kw,0.400000,0.130476,0.191111
4,filtered_TextRank_kw,0.170751,0.434286,0.242323
5,filtered_bert_kw,0.210000,0.159048,0.180000


Отфильтрованные по шаблонам ключевые слова показали улучшение метрик качества.  
У метода TextRank выше полнота, то есть он лучше всех выделяет ключевые слова, но при этом добавляет немало лишнего. 
У rake наилучший показатель по точности, следовательно этот метод лучше отсекает не